In [4]:
import pandas as pd

In [5]:
#!find . -maxdepth 3 -type f -exec du -h {} + | sort --human-numeric-sort --reverse
!free

             total       used       free     shared    buffers     cached
Mem:     261128012   23799516  237328496        116    1921652   18634588
-/+ buffers/cache:    3243276  257884736
Swap:            0          0          0


## INTEGRATE

In [6]:
blacklist=["abstract", "discussion", "background", "summary", "rationale", "introduction", "publisher", "introduction", "statement", "purpose", "objective" ]
blacklist += [t + ":" for t in blacklist]
def remove_special_words(text, blacklist=blacklist ):
    words = text.split(" ")
    try:
        first=words[0]
    except:
        return text
    
    first = first.strip().lower()
#     if any(first in string for string in blacklist):
#         return " ".join(words[1:]).strip()
    for w in reversed(blacklist): #prioritize "Abstract:The" replace over "AbstractThe"
        if w in first:
            return " ".join([first.split(w)[1].capitalize()] + words[1:]).strip()
    return text

remove_special_words("ABSTRACT   The northern corn rootworm"), remove_special_words("purpose:The northern corn rootworm")

('The northern corn rootworm', 'The northern corn rootworm')

In [10]:
df2 = pd.read_csv("downloads/train_june20_all.csv")

In [5]:
#df2[df2["text"].str.contains("Vietnam")]

In [11]:
df2["source"].unique()

array(['acl_cambridge', 'predicts_acl_pm2500_vocabed',
       'Oct1_clinical_studies_pm', 'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522', 'june152022_labels_envgeosoc',
       'june152022_labels_matscichem'], dtype=object)

In [13]:
#url0 = "https://docs.google.com/spreadsheets/d/1gqVPRvznJLZ1ori1qkBqx2xV8xPL1jLG_4WxaeewrzI/edit#gid=120391902"
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSrGaIXKQZOe0960YVYXn3z1aUpUNfsbYzFoUOv9-wGf84oLJCYr_KA2XzxeG0rUi0ff5lWCtHxmgsg/pub?output=csv"
#url1 = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSMOZaJsr2yRdX7SX5xLyvtd2fZXkj3DicRlE032sTbiBWinLtOGRX33eY6ewsm5jMv4xnsXMprHEqn/pub?output=csv"
#url2= "https://docs.google.com/spreadsheets/d/e/2PACX-1vSXaZ3xwCNvUTs3DgqzCaE3W481KJQhC6FqbNQ6zU04AGCj37vqS5TEMidyh46NC6UlyCbSXtj52UOc/pub?output=csv"
#url3 = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSkLXuuISZkDh53ttC9AZ1VvDaMXfqoOFSUqYHyzDHStlqNsbi3fiQoe993TSKoay5UyFHsG_otlCGO/pub?output=csv"

In [14]:
#source = "june152022_labels_envgeosoc"
source = "june152022_labels_matscichem"

In [17]:
df = pd.read_csv(url)
#df2 = pd.read_csv(url2)


In [18]:
df["labels"] = df["label"]
df["source"] = source
df = df.drop(columns=["predicts"])

In [19]:
df = df.dropna(subset=["labels"])
df["labels"] = df["labels"].astype(int) 
df = df[(df["labels"] == 1) | (df["labels"] == 0)] #fix typos in labels eg. misstyped 0 as 9


In [32]:
#df2 = pd.read_csv("downloads/43k_pm_bio.csv")
df3 = df2.append(df)
len(df3)

47446

In [33]:
df3 = df3.drop_duplicates(subset=["text"]) #doi with nans not good
len(df3)

44632

In [34]:
s2f = {
    "pm_METHODS": "Medicine",
    "pm_RESULTS": "Medicine",
    "pm_CONCLUSIONS": "Medicine",
    "pm_OBJECTIVE": "Medicine",
    "pm_OBJECTIVE": "Medicine",
    "a {negative_adjectives} ...  {problem_nouns}": "Medicine",
    "oct3_labels": "Medicine",
    " limitation|problem ... with ... l=be that": "Medicine",
    "labels_oct7": "Medicine",
    "Oct1_clinical_studies_pm": "Medicine",
    "predicts_acl_pm2500_vocabed": "Medicine",
    "Oct1_clinical_studies_pm": "Medicine",
    "acl_cambridge": "Computer Science",
    "psych1k_100522": "Psychology",
    "biology1k_050522": "Biology",   
}

df["fields2"] = df.source.map(lambda s: s2f.get(s)) 
df["fields"] = df.fields.combine_first(df['fields2'])

df["fields"]

0      Materials Science
1      Materials Science
2             Psychology
3              Chemistry
4              Chemistry
             ...        
745           Psychology
746    Materials Science
747           Psychology
748    Materials Science
749            Chemistry
Name: fields, Length: 750, dtype: object

In [35]:
df3 =df3[["text", "doi", "journalName", "title", "fields", "source", "labels"]]

In [37]:
df3["text"] = df3.text.map(remove_special_words)

In [38]:
df3.to_csv("downloads/train_june20_all.csv", index = False, header=True)

## TRAIN